In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

np.random.seed(42)

In [3]:
# Step 3A: Generate synthetic dataset

start_date = datetime(2025, 1, 1)
days = 30

campaigns = [
    {"campaign_id": "C1", "campaign_name": "Prospecting_US"},
    {"campaign_id": "C2", "campaign_name": "Retargeting_US"},
    {"campaign_id": "C3", "campaign_name": "Catalog_Sales_US"},
]

adsets = [
    {"adset_id": "AS1", "adset_name": "Broad_25_45", "campaign_id": "C1"},
    {"adset_id": "AS2", "adset_name": "Lookalike_1pct", "campaign_id": "C1"},
    {"adset_id": "AS3", "adset_name": "Site_Visitors", "campaign_id": "C2"},
    {"adset_id": "AS4", "adset_name": "Cart_Abandoners", "campaign_id": "C2"},
    {"adset_id": "AS5", "adset_name": "Catalog_Broad", "campaign_id": "C3"},
    {"adset_id": "AS6", "adset_name": "Catalog_Remarketing", "campaign_id": "C3"},
]

ads = []
for adset in adsets:
    for i in range(1, 4):
        ads.append({"ad_id": f"{adset['adset_id']}_AD{i}", "adset_id": adset["adset_id"]})

rows = []

for day in range(days):
    date = start_date + timedelta(days=day)
    
    for ad in ads:
        adset = next(a for a in adsets if a["adset_id"] == ad["adset_id"])
        campaign = next(c for c in campaigns if c["campaign_id"] == adset["campaign_id"])
        
        impressions = np.random.randint(4000, 10000)
        ctr = np.random.uniform(0.008, 0.015)
        
        if campaign["campaign_id"] == "C1" and day > 15:
            ctr *= 0.7  # Creative fatigue
        
        clicks = int(impressions * ctr)
        cpc = np.random.uniform(0.4, 0.9)
        spend = clicks * cpc
        
        conversion_rate = np.random.uniform(0.03, 0.06)
        if campaign["campaign_id"] == "C1" and day > 18:
            conversion_rate *= 0.6  # CPA spike
        
        conversions = max(1, int(clicks * conversion_rate))
        cpa = spend / conversions
        
        revenue_per_conversion = np.random.uniform(40, 80)
        if campaign["campaign_id"] == "C2" and day > 17:
            revenue_per_conversion *= 0.65  # ROAS drop
        
        revenue = conversions * revenue_per_conversion
        roas = revenue / spend
        
        if campaign["campaign_id"] == "C3" and day > 20:
            spend *= 0.6
            impressions = int(impressions * 0.6)  # Spend drop
        
        rows.append({
            "date": date.date(),
            "campaign_id": campaign["campaign_id"],
            "campaign_name": campaign["campaign_name"],
            "adset_id": adset["adset_id"],
            "adset_name": adset["adset_name"],
            "ad_id": ad["ad_id"],
            "spend": round(spend, 2),
            "impressions": impressions,
            "clicks": clicks,
            "ctr": round(ctr * 100, 2),
            "cpc": round(cpc, 2),
            "conversions": conversions,
            "cpa": round(cpa, 2),
            "roas": round(roas, 2)
        })

df = pd.DataFrame(rows)
df.head()

,date,campaign_id,campaign_name,adset_id,adset_name,ad_id,spend,impressions,clicks,ctr,cpc,conversions,cpa,roas
0,2025-01-01,C1,Prospecting_US,AS1,Broad_25_45,AS1_AD1,35.54,4860,45,0.93,0.79,2,17.77,3.25
1,2025-01-01,C1,Prospecting_US,AS1,Broad_25_45,AS1_AD2,64.98,9334,78,0.84,0.83,3,21.66,3.15
2,2025-01-01,C1,Prospecting_US,AS1,Broad_25_45,AS1_AD3,35.77,5685,47,0.84,0.76,2,17.88,2.24
3,2025-01-01,C1,Prospecting_US,AS2,Lookalike_1pct,AS2_AD1,46.38,9051,84,0.93,0.55,3,15.46,3.71
4,2025-01-01,C1,Prospecting_US,AS2,Lookalike_1pct,AS2_AD2,25.37,4474,54,1.23,0.47,2,12.68,4.31


In [4]:
# Step 3B: Observer – Campaign-level metric changes

df['date'] = pd.to_datetime(df['date'])
latest_date = df['date'].max()

last_7_start = latest_date - pd.Timedelta(days=6)
prev_7_start = latest_date - pd.Timedelta(days=13)
prev_7_end = latest_date - pd.Timedelta(days=7)

last_7 = df[(df['date'] >= last_7_start) & (df['date'] <= latest_date)]
prev_7 = df[(df['date'] >= prev_7_start) & (df['date'] <= prev_7_end)]

agg_last_7 = last_7.groupby(['campaign_id', 'campaign_name']).agg({
    'spend': 'sum', 'conversions': 'sum', 'cpa': 'mean', 'ctr': 'mean', 'roas': 'mean'
}).reset_index()

agg_prev_7 = prev_7.groupby(['campaign_id', 'campaign_name']).agg({
    'spend': 'sum', 'conversions': 'sum', 'cpa': 'mean', 'ctr': 'mean', 'roas': 'mean'
}).reset_index()

observer_df = agg_last_7.merge(agg_prev_7, on=['campaign_id', 'campaign_name'], suffixes=('_last_7', '_prev_7'))

for metric in ['spend', 'conversions', 'cpa', 'ctr', 'roas']:
    observer_df[f'{metric}_pct_change'] = (
        (observer_df[f'{metric}_last_7'] - observer_df[f'{metric}_prev_7']) 
        / observer_df[f'{metric}_prev_7'] * 100
    )

observer_df

,campaign_id,campaign_name,spend_last_7,conversions_last_7,cpa_last_7,ctr_last_7,roas_last_7,spend_prev_7,conversions_prev_7,cpa_prev_7,ctr_prev_7,roas_prev_7,spend_pct_change,conversions_pct_change,cpa_pct_change,ctr_pct_change,roas_pct_change
0,C1,Prospecting_US,1422.39,49,30.103095,0.771429,2.265000,1531.75,77,23.890238,0.828095,3.168810,-7.139546,-36.363636,26.005840,-6.843013,-28.522053
1,C2,Retargeting_US,2288.34,134,18.720238,1.157619,2.403095,2189.90,131,18.186429,1.100476,2.806905,4.495182,2.290076,2.935208,5.192557,-14.386292
2,C3,Catalog_Sales_US,1221.59,114,20.145238,1.126429,3.469048,1852.49,124,19.534524,1.141429,3.630238,-34.056864,-8.064516,3.126333,-1.314143,-4.440218


In [5]:
# Step 3C: Analyzer – Campaign-level diagnosis

analysis_results = []

for _, row in observer_df.iterrows():
    issues = []
    signals = []

    if row['ctr_pct_change'] < -15 and row['cpa_pct_change'] > 20:
        issues.append("Creative fatigue")
        signals.append(f"CTR decreased by {row['ctr_pct_change']:.1f}%")
        signals.append(f"CPA increased by {row['cpa_pct_change']:.1f}%")
    elif row['cpa_pct_change'] > 20 and abs(row['ctr_pct_change']) <= 10:
        issues.append("Efficiency decline")
        signals.append(f"CPA increased by {row['cpa_pct_change']:.1f}%")
        signals.append("CTR relatively stable")
    
    if row['roas_pct_change'] < -25 and abs(row['cpa_pct_change']) <= 10:
        issues.append("ROAS degradation")
        signals.append(f"ROAS decreased by {row['roas_pct_change']:.1f}%")
        signals.append("CPA relatively stable")
    
    if row['spend_pct_change'] < -30 and abs(row['ctr_pct_change']) <= 10:
        issues.append("Delivery issue")
        signals.append(f"Spend decreased by {row['spend_pct_change']:.1f}%")
        signals.append("CTR relatively stable")
    
    if not issues:
        issues.append("No major issue detected")
        signals.append("Metric changes within normal range")
    
    analysis_results.append({
        "campaign_id": row['campaign_id'],
        "campaign_name": row['campaign_name'],
        "issue_detected": ", ".join(issues),
        "signals": signals
    })

analysis_df = pd.DataFrame(analysis_results)
analysis_df

,campaign_id,campaign_name,issue_detected,signals
0,C1,Prospecting_US,Efficiency decline,"[CPA increased by 26.0%, CTR relatively stable]"
1,C2,Retargeting_US,No major issue detected,[Metric changes within normal range]
2,C3,Catalog_Sales_US,Delivery issue,"[Spend decreased by -34.1%, CTR relatively sta..."


In [6]:
# Step 3D: Explainer – GenAI prompt construction

def build_explainer_prompt(row):
    signals_text = "\n".join([f"- {s}" for s in row['signals']])
    
    prompt = f"""
Campaign name: {row['campaign_name']}

Detected issue(s): {row['issue_detected']}

Supporting signals:
{signals_text}

Instructions:
Explain the issue in 2–3 sentences.
Use cautious, professional language.
Do not speculate beyond the signals provided.
"""
    return prompt

analysis_df['explainer_prompt'] = analysis_df.apply(build_explainer_prompt, axis=1)
analysis_df[['campaign_name', 'explainer_prompt']]

,campaign_name,explainer_prompt
0,Prospecting_US,\nCampaign name: Prospecting_US\n\nDetected is...
1,Retargeting_US,\nCampaign name: Retargeting_US\n\nDetected is...
2,Catalog_Sales_US,\nCampaign name: Catalog_Sales_US\n\nDetected ...


In [7]:
# Step 3E: Recommender – Map issue to safe actions

def map_issue_to_action(issue):
    mapping = {
        "Creative fatigue": "Refresh creatives, rotate ad formats, test new copy/images.",
        "Efficiency decline": "Review bidding strategy and adjust budget allocation.",
        "ROAS degradation": "Evaluate traffic quality and optimize targeting or landing page.",
        "Delivery issue": "Check bid caps, audience saturation, and budget allocation.",
        "No major issue detected": "Continue monitoring; no immediate action needed."
    }
    actions = []
    for single_issue in issue.split(", "):
        if single_issue in mapping:
            actions.append(mapping[single_issue])
    return " ".join(actions)

analysis_df['recommended_action'] = analysis_df['issue_detected'].apply(map_issue_to_action)
analysis_df[['campaign_name', 'issue_detected', 'recommended_action']]

,campaign_name,issue_detected,recommended_action
0,Prospecting_US,Efficiency decline,Review bidding strategy and adjust budget allo...
1,Retargeting_US,No major issue detected,Continue monitoring; no immediate action needed.
2,Catalog_Sales_US,Delivery issue,"Check bid caps, audience saturation, and budge..."
